In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [2]:
def display_topics(model, feature_names, no_top_words):
  for topic_idx, topic in enumerate(model.components_):
    print("Topic %d:" % (topic_idx))
    print(" ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [5]:
with open("tokens_2020_1250000.txt", "r") as text_file:
  documents = text_file.readlines()[0].split("<s>")

In [7]:
no_features = 1000

# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words='english')
tf = tf_vectorizer.fit_transform(documents)
tf_feature_names = tf_vectorizer.get_feature_names_out()

In [10]:
no_topics = 10

# Run LDA
lda = LatentDirichletAllocation(n_components=no_topics, max_iter=5, learning_method='online',
                                learning_offset=50.,random_state=0).fit(tf)

no_top_words = 10
display_topics(lda, tf_feature_names, no_top_words)

Topic 0:
black video twitter media series said white people social lives
Topic 1:
family harry years prince getty daughter children father royal husband
Topic 2:
year new seattle 10 cent 000 run high 20 energy
Topic 3:
trump president election biden house said party donald white political
Topic 4:
police said man officers death floyd year officer incident mr
Topic 5:
said state people government public south home department county officials
Topic 6:
said year new government world china company minister business million
Topic 7:
said people time like just home year uk going know
Topic 8:
season game league team players year club united time said
Topic 9:
coronavirus covid said health 19 people cases virus new pandemic
